In [2]:
import pandas as pd

# load data into a pandas DataFrame
df = pd.read_csv('EFIplus_medit.zip',compression='zip', sep=";")

# calculate species richness for each sampling site
species_richness = df.sum(axis=1)

# create a new DataFrame with sampling sites and species richness
richness_df = pd.DataFrame({'sampling_site': df.index, 'species_richness': species_richness})

# print the new DataFrame
print(richness_df)


      sampling_site  species_richness
0                 0       1553.483016
1                 1       3626.131334
2                 2       3532.207657
3                 3       3386.572800
4                 4       3309.326501
...             ...               ...
5006           5006       2069.477006
5007           5007       2247.059872
5008           5008       2154.291358
5009           5009       2981.893730
5010           5010       2459.108881

[5011 rows x 2 columns]


C:\Users\Miguel Paulo\AppData\Local\Temp\ipykernel_8324\2107350845.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  species_richness = df.sum(axis=1)


In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import shapiro

# load data into a pandas DataFrame
data = pd.read_csv('EFIplus_medit.zip',compression='zip', sep=";")

# check for normality of the continuous variables using Shapiro-Wilk test
for col in ['Altitude', 'Actual_river_slope', 'Elevation_mean_catch', 'prec_ann_catch', 'temp_ann', 'temp_jan', 'temp_jul']:
    stat, p = shapiro(data[col])
    print(col, 'p-value:', p)

# transform variables as necessary based on Shapiro-Wilk test
data['Altitude'] = np.log(data['Altitude'])
data['Actual_river_slope'] = np.sqrt(data['Actual_river_slope'])
data['prec_ann_catch'] = np.sqrt(data['prec_ann_catch'])
data['temp_ann'] = np.sqrt(data['temp_ann'])
data['temp_jan'] = np.sqrt(data['temp_jan'])
data['temp_jul'] = np.sqrt(data['temp_jul'])

# perform linear regression for each variable
for col in ['Altitude', 'Actual_river_slope', 'Elevation_mean_catch', 'prec_ann_catch', 'temp_ann', 'temp_jan', 'temp_jul']:
    X = data[col]
    y = data['species_richness']
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    print(col, 'coefficient:', model.params[col], 'r-square:', model.rsquared, 'F-statistic:', model.fvalue, 'p-value:', model.f_pvalue)


C:\Users\Miguel Paulo\AppData\Roaming\Python\Python310\site-packages\scipy\stats\_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
C:\Users\Miguel Paulo\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Miguel Paulo\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


Altitude p-value: 2.802596928649634e-45
Actual_river_slope p-value: 1.0
Elevation_mean_catch p-value: 1.0
prec_ann_catch p-value: 1.0
temp_ann p-value: 1.0
temp_jan p-value: 1.0
temp_jul p-value: 1.0


KeyError: 'species_richness'